# About: 検証環境の削除

---

コンテナのパッケージアップデートやMoodleのバージョンアップなどの手順を中止し検証環境を削除します。

## 概要

検証環境を削除します。このNotebookを実行することで、元の運用環境だけを実行する状態に戻ります。

![検証環境の削除](images/moodle-191-01.png)

### グループ名の指定

このNotebookの操作対象となる UnitGroup名を指定します。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name =

#### チェック

指定された `UnitGroup`名に対応する group_varsファイルが存在していることを確認します。エラーになる場合は、指定したUnitGroup名が正しくないか、「010-パラメータ設定.ipynb」のNotebookを実行していないことが考えられます。

In [ ]:
from pathlib import Path
if not (Path('group_vars') / (ugroup_name + '.yml')).exists():
    raise RuntimeError(f"ERROR: not exists {ugroup_name + '.yml'}")

UnitGroupに属する VCノードに対して Ansible で操作できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping
!ansible {ugroup_name} -b -a 'whoami'

検証環境に関するパラメータが group_vars に保存されていることを確認します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

if 'update_project_tag' not in gvars:
    raise RuntimeError("ERROR")

### VCCアクセストークンの入力

VCCにアクセスするためのトークンを入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

#### チェック

入力されたアクセストークンが正しいことを、VCCにアクセスして確認します。

In [ ]:
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

### 準備

これまでに他のNotebookで設定したパラメータを読み込む処理などを行います。

group_varsファイルに保存されているパラメータを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

VCCのVault サーバにアクセスする際に必要となるパラメータを環境変数に設定します。

In [ ]:
import os
os.environ['VAULT_ADDR'] = vcp.vcc_info()['vault_url']
os.environ['VAULT_TOKEN'] = vcc_access_token

## リバースプロキシの設定

検証用のアドレスからのアクセスも元の運用環境にアクセスできるようにリバースプロキシの設定を変更します。

![リバースプロキシの変更](images/moodle-191-02.png)

現在のリバースプロキシの設定ファイルを確認します。

In [ ]:
!ansible {ugroup_name} -a 'cat /opt/moodle/proxy/conf/moodle-proxy.conf'

次のセルを実行すると検証環境用設定を削除したApache HTTP Serverの設定ファイル
`moodle-proxy.conf`をローカル環境に作成します。作成後に現在の設定ファイルとの差分を表示します。
また、ローカル環境に生成した `moodle-proxy.conf` を編集するためのリンクを最後に表示します。

In [ ]:
%run scripts/edit_conf.py
update_proxy_conf(ugroup_name, extra_vars={'clear_test_env': 'yes'})

ローカル環境の変更後の `moodle-proxy.conf` をVCノードに配置します。
その後、設定ファイルの記述内容を反映させるためにリバースプロキシコンテナを再起動します。

In [ ]:
apply_proxy_conf(ugroup_name)

再起動後のコンテナの状態を確認します。`proxy`コンテナの状態が `Up` になっていることを確認してください。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose ps'

## 検証環境のコンテナを停止する

![検証環境用コンテナの停止](images/moodle-191-03.png)

検証環境のコンテナを停止する前のコンテナ実行状態を確認します。

In [ ]:
!ansible {ugroup_name} -a 'docker ps'

`docker-compose.yml`から検証環境に関するコンテナを削除します。

次のセルを実行すると検証環境用コンテナを削除した `docker-compose.yml` をローカル環境に作成し更新前との差分を表示します。また最後に表示されたリンクから更新後の `docker-compose.yml` を編集することもできます。

In [ ]:
%run scripts/edit_conf.py
update_docker_compose(ugroup_name, extra_vars={'clear_test_env': 'yes'})

ローカル環境の`docker-compose.yml`をVCノードに配置し、検証環境のコンテナを停止します。

In [ ]:
upload_docker_compose(ugroup_name, apply=True)

検証環境のコンテナを停止したことを確認するために、コンテナの実行状態を表示します。

In [ ]:
!ansible {ugroup_name} -a 'docker ps'

## 検証環境を削除する

![検証環境を削除](images/moodle-191-04.png)

### 検証環境の論理ボリュームを削除する

現在の論理ボリュームを一覧表示します。

In [ ]:
!ansible {ugroup_name} -b -a 'lvs -S "lv_attr=~V" -O lv_name'

削除対象となる論理ボリュームを表示します。

In [ ]:
lvol_map = {'moodle': 'moodle', 'php': 'moodle', 'db': 'db'}
tag = gvars['update_project_tag']
remove_vols = dict([(f'{tag}_{v}_test', vg) for v, vg in lvol_map.items()])

for lv, vg in remove_vols.items():
    print(f'lv={lv} vg={vg}')

論理ボリュームを削除します。

In [ ]:
for lv, vg in remove_vols.items():
    print(f'lv={lv} vg={vg}')
    name = f'{vg}-{lv}{"-enc" if gvars[vg + "_volume_encrypt"] else ""}'
    !ansible {ugroup_name} -b -a 'umount /dev/mapper/{name}'
    if gvars[vg + "_volume_encrypt"]:
        !ansible {ugroup_name} -b -a 'cryptsetup close {name}'
    !ansible {ugroup_name} -b -m lvol -a 'vg={vg} lv={lv} state=absent force=yes'

削除後の論理ボリュームの一覧を表示します。

In [ ]:
!ansible {ugroup_name} -b -a 'lvs -S "lv_attr=~V" -O lv_name'

### 検証環境の設定ファイルを削除する

検証環境の設定ファイルの状況を確認します。

In [ ]:
target_dirs = ' '.join([f'{x}test-{tag}' for x in ['moodle', 'db']])
print(target_dirs)
!ansible {ugroup_name} -a 'chdir=/opt/moodle tree {target_dirs}'

検証環境の設定ファイルを削除します。

In [ ]:
!ansible {ugroup_name} -b -a 'chdir=/opt/moodle rm -rf {target_dirs}'

削除されたことを確認します。

> 削除されていれば`[error opening dir]`と表示されます。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle tree {target_dirs}'

### 検証環境のコンテナイメージを削除する

検証環境で利用していたコンテナイメージの一覧を表示します。

In [ ]:
target_image = f'local/{gvars["update_container_target"]}:{gvars["update_project_tag"]}-test'
print(target_image)
!ansible {ugroup_name} -a 'docker images {target_image}'

検証環境のコンテナイメージを削除します。

In [ ]:
!ansible {ugroup_name} -a 'docker rmi {target_image}'

コンテナイメージが削除されたことを確認します。

> 削除が成功していれば、ヘッダーのみが表示されます。

In [ ]:
!ansible {ugroup_name} -a 'docker images {target_image}'

### group_varsの更新

検証環境に関するパラメータを `group_vars`ファイルから削除します。

`group_vars`に保存されている検証環境に関するパラメータ名を確認します。

In [ ]:
target_names = [x for x in gvars.keys() if x.startswith('update_')]
print(target_names)

`group_vars`から上のセルで確認したパラメータを削除します。

In [ ]:
remove_group_vars(
    ugroup_name,
    *target_names,
)

## Moodle を利用できることを確認

Moodle環境にアクセスして正しく動作していることを確認します。

次のセルを実行するとMoodleのアドレスが表示されます。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}" target="_blank">{0}</a>'.format(gvars['moodle_url']))